In [1]:
from huggingface_hub import notebook_login
from datasets import load_dataset

notebook_login()

In [2]:
translated_dataset_hf = load_dataset("csv",data_files="./final_combined.csv")

In [3]:
translated_dataset_hf

DatasetDict({
    train: Dataset({
        features: ['original_instruction', 'original_output', 'translated_instruction', 'translated_output'],
        num_rows: 389608
    })
})

In [4]:
import pandas as pd
df = pd.read_csv('./final_combined.csv')
# print(df.to_string()) 

In [5]:
df.describe()

,original_instruction,original_output,translated_instruction,translated_output
count,389543,389523,389608,389608
unique,362776,333366,328492,334473
top,I don't care.,Negative,ಸಾರ್ವಜನಿಕವಾಗಿ ಮಾತನಾಡುವ ನನ್ನ ಕೌಶಲ್ಯವನ್ನು ನಾನು ಹ...,?
freq,58,90,70,664


In [6]:
# Create the first section (en_to_en)
section1 = df[['original_instruction', 'original_output']].rename(columns={'original_instruction': 'instruction', 'original_output': 'output'})
section1['translation'] = 'en_to_en'

# Create the second section (kn_to_kn)
section2 = df[['translated_instruction', 'translated_output']].rename(columns={'translated_instruction': 'instruction', 'translated_output': 'output'})
section2['translation'] = 'kn_to_kn'

# Create the third section (en_to_kn)
section3 = df[['original_instruction', 'translated_output']].rename(columns={'original_instruction': 'instruction', 'translated_output': 'output'})
section3['translation'] = 'en_to_kn'

# Create the fourth section (kn_to_en)
section4 = df[['translated_instruction', 'original_output']].rename(columns={'translated_instruction': 'instruction', 'original_output': 'output'})
section4['translation'] = 'kn_to_en'

# Concatenate all sections to create the final DataFrame
final_df = pd.concat([section1, section2, section3, section4], ignore_index=True)

# Print the description of the final DataFrame
print(final_df.describe())

                                              instruction   output translation
count                                             1558302  1558262     1558432
unique                                             691261   667235           4
top     ಸಾರ್ವಜನಿಕವಾಗಿ ಮಾತನಾಡುವ ನನ್ನ ಕೌಶಲ್ಯವನ್ನು ನಾನು ಹ...        ?    en_to_en
freq                                                  140     1328      389608


In [7]:
import pandas as pd
import random

# Assuming 'final_df' is your DataFrame after the previous operations

# Filter rows with 'kn_to_en' translation
en_to_kn_rows = final_df[final_df['translation'] == 'en_to_kn']

# Phrases to be added
phrases = [
    "respond to the following in kannada",
    "answer in kannada",
    "tell it to me in kannada",
    "respond in kannada",
    "reply in kannada",
]

# Function to randomly choose a phrase
def get_random_phrase():
    return random.choice(phrases)

# Apply the random phrase to each row either at the beginning or end
en_to_kn_rows['instruction'] = en_to_kn_rows.apply(lambda row: f"{get_random_phrase()} {row['instruction']}" if random.choice([True, False]) else f"{row['instruction']} {get_random_phrase()}", axis=1)

# Print the modified DataFrame
print(en_to_kn_rows)

# Replace modified rows in the original DataFrame
final_df.loc[en_to_kn_rows.index, 'instruction'] = en_to_kn_rows['instruction']

# Print the updated DataFrame
print(final_df)

                                               instruction  \
779216   Give three tips for staying healthy. answer in...   
779217   answer in kannada What are the three primary c...   
779218   respond to the following in kannada Describe t...   
779219   How can we reduce air pollution? answer in kan...   
779220   answer in kannada Describe a time when you had...   
...                                                    ...   
1168819  answer in kannada Compare and contrast regulat...   
1168820  Reorganize the following paragraph into a cohe...   
1168821  answer in kannada Given a list of cities, clas...   
1168822  What is the meaning of the phrase "never judge...   
1168823  Edit the following sentence so it reads correc...   

                                                    output translation  
779216   1.Eat ಸಮತೋಲಿತ ಆಹಾರ ಮತ್ತು ಸಾಕಷ್ಟು ಹಣ್ಣುಗಳು ಮತ್ತ...    en_to_kn  
779217       ಮೂರು ಪ್ರಾಥಮಿಕ ಬಣ್ಣಗಳು ಕೆಂಪು, ನೀಲಿ ಮತ್ತು ಹಳದಿ.    en_to_kn  
779218   ಪರಮಾಣು ಪ್ರೋಟಾನ್ಗಳು ಮತ್ತು ನ್

C:\Users\Adithya\AppData\Local\Temp\ipykernel_13048\2990095260.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  en_to_kn_rows['instruction'] = en_to_kn_rows.apply(lambda row: f"{get_random_phrase()} {row['instruction']}" if random.choice([True, False]) else f"{row['instruction']} {get_random_phrase()}", axis=1)


In [8]:
# Filter rows with 'kn_to_en' translation
kn_to_en_rows = final_df[final_df['translation'] == 'kn_to_en']

# Phrases to be added for English response
english_phrases = [
    "respond to the following in English",
    "answer in English",
    "tell it to me in English",
    "respond in english"
    "reply in english"
]

# Function to randomly choose an English phrase
def get_random_english_phrase():
    return random.choice(english_phrases)

# Apply the random English phrase to each row either at the beginning or end
kn_to_en_rows['instruction'] = kn_to_en_rows.apply(lambda row: f"{get_random_english_phrase()} {row['instruction']}" if random.choice([True, False]) else f"{row['instruction']} {get_random_english_phrase()}", axis=1)

# Print the modified DataFrame
print(kn_to_en_rows)

# Replace modified rows in the original DataFrame
final_df.loc[kn_to_en_rows.index, 'instruction'] = kn_to_en_rows['instruction']

# Print the updated DataFrame
print(final_df)

                                               instruction  \
1168824  answer in English ಆರೋಗ್ಯವಾಗಿರಲು ಮೂರು ಸಲಹೆಗಳನ್ನ...   
1168825  respond in englishreply in english ಮೂರು ಪ್ರಾಥಮ...   
1168826  ಪರಮಾಣುವಿನ ರಚನೆಯನ್ನು ವಿವರಿಸಿ. respond to the fo...   
1168827  respond in englishreply in english ವಾಯು ಮಾಲಿನ್...   
1168828  ನೀವು ಕಠಿಣ ನಿರ್ಧಾರವನ್ನು ತೆಗೆದುಕೊಳ್ಳಬೇಕಾದ ಸಮಯವನ್...   
...                                                    ...   
1558427  respond in englishreply in english ನಿಯಂತ್ರಣ ಮತ...   
1558428  answer in English ಈ ಕೆಳಗಿನ ಪ್ಯಾರಾಗ್ರಾಫ್ ಅನ್ನು ...   
1558429  answer in English ನಗರಗಳ ಪಟ್ಟಿಯನ್ನು ನೀಡಿದರೆ, ಪ್...   
1558430  "ಎಂದಿಗೂ ಪುಸ್ತಕವನ್ನು ಅದರ ಮುಖಪುಟದಿಂದ ನಿರ್ಣಯಿಸಬೇಡ...   
1558431  ಈ ಕೆಳಗಿನ ವಾಕ್ಯವು ಸರಿಯಾಗಿ ಓದುವಂತೆ ಅದನ್ನು ಸಂಪಾದಿ...   

                                                    output translation  
1168824  1.Eat a balanced diet and make sure to include...    kn_to_en  
1168825  The three primary colors are red, blue, and ye...    kn_to_en  
1168826  An atom is made up of a nuc

C:\Users\Adithya\AppData\Local\Temp\ipykernel_13048\1086833397.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kn_to_en_rows['instruction'] = kn_to_en_rows.apply(lambda row: f"{get_random_english_phrase()} {row['instruction']}" if random.choice([True, False]) else f"{row['instruction']} {get_random_english_phrase()}", axis=1)


In [9]:
final_df['text'] = final_df.apply(lambda row: f"<|user|>{row['instruction']}<|endoftext|><|assistant|>{row['output']}<|endoftext|>", axis=1)

In [10]:
final_df.to_csv('./bilingual_instruct.csv', index=False)

In [11]:
bilingual_dataset_hf = load_dataset("csv",data_files="./bilingual_instruct.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
bilingual_dataset_hf

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'translation', 'text'],
        num_rows: 1558432
    })
})

In [13]:
bilingual_dataset_hf.push_to_hub("CognitiveLab/bilingual_instruct_kn")

Uploading the dataset shards:   0%|          | 0/5 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/312 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/312 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/312 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/312 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/312 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/CognitiveLab/bilingual_instruct_kn/commit/554cbd730938f7336d68fd9758660f36e12afac4', commit_message='Upload dataset', commit_description='', oid='554cbd730938f7336d68fd9758660f36e12afac4', pr_url=None, pr_revision=None, pr_num=None)